In [1]:
import os, sys, math, io
import numpy as np
import pandas as pd
import multiprocessing as mp
import bson
import struct
from PIL import Image
import time
import shutil

%matplotlib inline
import matplotlib.pyplot as plt

from collections import defaultdict
from tqdm import *

In [2]:
# loading PyTorch
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.utils.data import Dataset

In [3]:
# load pre-processing study results to build hash
RawDataStudy_dir = '/media/hua/HuaSSD/KaggleData/CdiscountImageClassificationChallenge/RawDataStudy/'
train_offsets_df = pd.read_csv( RawDataStudy_dir + 'train_offsets.csv', index_col=0)
train_images_df = pd.read_csv( RawDataStudy_dir + 'train_freqs.csv')
val_images_df = pd.read_csv( RawDataStudy_dir + 'val_images_all.csv', index_col=0)
categories_df = pd.read_csv( RawDataStudy_dir + 'categories_name_to_id.csv')

/home/hua/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
train_images_df.head()

,product_id,category_idx,img_idx,freqs
0,0,5055,0,61688
1,1,5055,0,61688
2,5,5055,0,61688
3,11,5055,0,61688
4,16,5055,0,61688


In [5]:
train_images_df.tail()

,product_id,category_idx,img_idx,freqs
12129136,19028368,5101,2,13
12129137,20643558,5101,0,13
12129138,20643558,5101,1,13
12129139,20643558,5101,2,13
12129140,20643558,5101,3,13


In [6]:
# build hash map for l1 and l2 id
idx2l1 = list(categories_df['category_level1'])
idx2l2 = list(categories_df['category_level2'])
frequencies = train_offsets_df['category_id'].value_counts()

In [7]:
data_dir = "/media/hua/HuaSSD/KaggleData/CdiscountImageClassificationChallenge/"
train_bson_path = os.path.join(data_dir, "train.bson")
train_bson_file = open(train_bson_path, "rb")

In [8]:
class BSONIterator(Dataset):
    def __init__(self, bson_file, images_df, offsets_df, transform, mode = 'train'):
        super(BSONIterator, self).__init__()
        self.file = bson_file
        self.images_df = images_df
        self.offsets_df = offsets_df
        self.transform = transform
        self.mode = mode

    def __getitem__(self, idx):
        image_row = self.images_df.iloc[idx]
        product_id = image_row["product_id"]
        offset_row = self.offsets_df.loc[product_id]
        # Random access this product's data from the BSON file.
        self.file.seek(offset_row["offset"])
        item_data = self.file.read(offset_row["length"])
        # Grab the image from the product.
        item = bson.BSON.decode(item_data)
        img_idx = image_row["img_idx"]
        bson_img = item["imgs"][img_idx]["picture"]

        # Load the image.
        image = io.BytesIO(bson_img)
        img = Image.open(image)
        x = self.transform(img)
        idx = int(image_row["category_idx"])
        level1 = int(idx2l1[idx])
        level2 = int(idx2l2[idx])

        target1 = torch.LongTensor([level1])
        target2 = torch.LongTensor([level2])
        target3 = torch.LongTensor([idx])
        if self.mode == 'train':
            return x, target1, target2, target3 #for the sake of pin_memory and async
        if self.mode == 'valid':
            return x, [target1]*10, [target2]*10, [target3]*10
        else:
            return x
    
    def __len__(self):
        return len(self.images_df)

In [9]:
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
transform_train = T.Compose([T.RandomHorizontalFlip(), 
                             T.ToTensor(),T.Normalize(mean=mean, std=std)])
transform_val = T.Compose([T.ToTensor(),T.Normalize(mean=mean, std=std)])

Create a generator for training and a generator for validation.

In [10]:
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df, transform_train, mode = 'train')
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df, transform_val, mode = 'train')

In [11]:
print(len(train_gen), len(val_gen))

12129141 242152


In [12]:
# cutoff = 1000
# freqs = train_images_df['freqs']
# weights = np.where(freqs>cutoff, 1.0, 2.0)
# num_samples = len(train_gen)

In [12]:
batch_size = 256
loader_train = DataLoader(train_gen, batch_size=batch_size, 
                          sampler=sampler.RandomSampler(train_gen), 
                          num_workers=1, pin_memory = True)
loader_val = DataLoader(val_gen, batch_size=batch_size, sampler=sampler.SequentialSampler(val_gen), 
                        num_workers=1, pin_memory = True)

In [13]:
print(len(loader_train), len(loader_val))

47380 946


In [14]:
# itr = iter(loader_train)

In [15]:
# img, target1, target2, target3 = next(itr)

In [16]:
# img.size(), target1.size(), target2.size(), target3.size()

In [17]:
# itr = iter(loader_val)

In [18]:
# img, target1, target2, target3 = next(itr)

In [19]:
# img.size(), target1.size(), target2.size(), target3.size()

# Training

**ResNet101**

In [20]:
model = torchvision.models.resnet101(pretrained=True)
#0.5 comparable to the tencrop method with 4X4/6X6
model.avgpool = nn.Sequential(nn.AvgPool2d(kernel_size = 6), nn.Dropout(p=0.2,inplace=True))
model.fc = nn.Linear(in_features=2048, out_features=49 + 483 + 5270)

In [21]:
# loading models ...
model_dir = '/media/hua/HuaSSD/KaggleData/CdiscountImageClassificationChallenge/ModelResNet/' #model saved in SSD
# em... we need to change this from time to time
#trained_model = model_dir + 'ResNet101_L4L5_4Epoch_DO05_W111_20171204.pth.tar'
#trained_model = model_dir + 'ResNet101_L4L5_6Epoch_lrneg3_DO05_W111_20171208.pth.tar'
trained_model = model_dir + 'ResNet101_L4L5_7Epoch_lrneg4_DO01_W111_20171210.pth.tar'

def load_model(model, trained_model):
    if os.path.isfile(trained_model):
        print("=> loading checkpoint '{}'".format(trained_model))
        checkpoint = torch.load(trained_model)
        model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint '{}'".format(trained_model))
        return model
    else:
        print("=> no checkpoint found at '{}'".format(best_model))

In [22]:
model = load_model(model, trained_model)

=> loading checkpoint '/media/hua/HuaSSD/KaggleData/CdiscountImageClassificationChallenge/ModelResNet/ResNet101_L4L5_7Epoch_lrneg4_DO01_W111_20171210.pth.tar'
=> loaded checkpoint '/media/hua/HuaSSD/KaggleData/CdiscountImageClassificationChallenge/ModelResNet/ResNet101_L4L5_7Epoch_lrneg4_DO01_W111_20171210.pth.tar'


In [23]:
for layer in [model.conv1, model.bn1, model.relu, model.maxpool, model.layer1, model.layer2, model.layer3]:
    for param in layer.parameters():
        param.requires_grad = False
model.cuda()

ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): Bottleneck (
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (downsample): Sequential (
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (1): Bott

In [24]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(lr, optimizer, epoch, denominator = 2):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = lr * (0.1 ** (epoch // denominator))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    batch_size = target.size(0)
    _, pred = output.max(dim=1)
    correct = pred.eq(target)
    res = []
    for k in topk:
        correct_k = correct.float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [25]:
def train(train_loader, model, criterion, optimizer, weights, epoch, print_freq = 50):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    loss_log = []
    acc_log = []
    PREC1 = 1

    # switch to train mode
    model.train()

    end = time.time()
    for i, (img, target1, target2, target3) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        
        img = img.cuda(async=True)
        img_var = Variable(img)
        
        target1 = target1.view(-1).cuda(async=True)
        target1_var = Variable(target1)
        target2 = target2.view(-1).cuda(async=True)
        target2_var = Variable(target2)
        target3 = target3.view(-1).cuda(async=True)
        target3_var = Variable(target3)

        # compute output
        output = model(img_var)
        loss1 = criterion(output[:, :49], target1_var)
        loss2=  criterion(output[:, 49:532], target2_var)
        loss3=  criterion(output[:, 532:], target3_var)
        loss = loss1*weights[0] + loss2*weights[1] + loss3*weights[2]
        # measure accuracy and record loss of selected target
        if weights[2]>0:
            prec1 = accuracy(output.data[:, 532:], target3, topk=(1, ))[0]#only need top1
        elif weights[1]>0:
            prec1 = accuracy(output.data[:, 49:532], target2, topk=(1, ))[0]
        else:
            prec1 = accuracy(output.data[:, :49], target1, topk=(1, ))[0]
        losses.update(loss.data[0], img.size(0)) #[0] to take out the float inside torch.Tensor
        top1.update(prec1[0], img.size(0))
        loss_log.append(losses.val)
        acc_log.append(top1.val)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))
    return loss_log, acc_log

In [26]:
def validate(val_loader, model, weights, print_freq=20):
    batch_time = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (img, target1, target2, target3) in enumerate(val_loader):
        
        img = img.cuda(async=True)
        img_var = Variable(img, volatile=True)
        
        target1 = target1.view(-1).cuda(async=True)
        target1_var = Variable(target1)
        target2 = target2.view(-1).cuda(async=True)
        target2_var = Variable(target2)
        target3 = target3.view(-1).cuda(async=True)
        target3_var = Variable(target3)

        # compute output
        output = model(img_var)

        # measure accuracy and record loss
        if weights[2]> 0:
            prec1 = accuracy(output.data[:, 532:], target3, topk=(1, ))[0]#only need top1
        elif weights[1]>0:
            prec1 = accuracy(output.data[:, 49:532], target2, topk=(1, ))[0]
        else:
            prec1 = accuracy(output.data[:, :49], target1, topk=(1, ))[0]
        top1.update(prec1[0], img.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   i, len(val_loader), batch_time=batch_time, top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return top1.avg

In [27]:
def validate_average(val_loader, model, weights, n = 10, print_freq=80):
    batch_time = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (img, target1, target2, target3) in enumerate(val_loader):
        
        img = img.cuda(async=True)
        img_var = Variable(img, volatile=True)
        
        target1 = target1.view(-1, n)[:, 0]
        target1 = target1.cuda()
        target1_var = Variable(target1)
        target2 = target2.view(-1, n)[:, 0]
        target2 = target2.cuda()
        target2_var = Variable(target2)
        target3 = target3.view(-1, n)[:, 0]
        target3 = target3.cuda()
        target3_var = Variable(target3)

        # compute output
        output = model(img_var)
        output = output.view(-1, n, 5802).mean(dim = 1)

        # measure accuracy and record loss
        if weights[2]> 0:
            prec1 = accuracy(output.data[:, 532:], target3, topk=(1, ))[0]#only need top1
        elif weights[1]>0:
            prec1 = accuracy(output.data[:, 49:532], target2, topk=(1, ))[0]
        else:
            prec1 = accuracy(output.data[:, :49], target1, topk=(1, ))[0]
        top1.update(prec1[0], img.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   i, len(val_loader), batch_time=batch_time, top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return top1.avg

In [29]:
if __name__ == '__main__':
    best_prec1 = 65
    criterion = nn.CrossEntropyLoss().cuda()
    #lr = 1e-2
    #reduce learning rate when model is well trained
    #lr = 1e-3
    #reduce learning rate agian...
    #lr = 1e-4
    #reduce learning rate agian...
    lr = 1e-5    
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr = lr, momentum=0.9, 
                          weight_decay=0)
    resume = None
    # em... train several epochs
    start_epoch = 0
    epochs = 3
    arch = 'resnet101_levelID'

    for epoch in range(start_epoch, epochs):
        adjust_learning_rate(lr=lr, optimizer=optimizer, epoch=epoch, denominator=2)

        # train for one epoch
        for weights in [[1, 1, 1]]:
            loss_log, acc_log = train(train_loader=loader_train, model=model, criterion=criterion,
                                      weights = weights, optimizer=optimizer, epoch=epoch)

        # evaluate on validation set
            prec1 = validate(val_loader=loader_val, model=model, weights=weights)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': arch,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer' : optimizer.state_dict(),
        }, is_best)

        #plot loss and acc
        #fig = plt.figure(figsize = (6,3), dpi = 600)
        #loss_log = np.array(loss_log)
        #ax1 = plt.subplot(121)
        #ax1.plot(loss_log)
        #ax1.set_ylabel('Loss', weight = 'bold')
        #acc_log = np.array(acc_log)
        #ax2 = plt.subplot(111)
        #ax2.plot(acc_log)
        #ax2.set_ylabel('Train_accuracy', weight = 'bold')
        #np.savetxt(X=np.vstack((loss_log, acc_log)), fname='loss_acc_log.txt', fmt='%.3f')

Epoch: [0][0/47380]	Time 28.829 (28.829)	Data 1.447 (1.447)	Loss 2.7658 (2.7658)	Prec@1 64.453 (64.453)
Epoch: [0][50/47380]	Time 1.709 (2.235)	Data 0.000 (0.029)	Loss 2.6090 (2.7506)	Prec@1 66.797 (67.096)
Epoch: [0][100/47380]	Time 1.708 (1.973)	Data 0.000 (0.015)	Loss 2.2730 (2.7353)	Prec@1 71.484 (67.358)
Epoch: [0][150/47380]	Time 1.709 (1.886)	Data 0.000 (0.010)	Loss 2.8905 (2.7365)	Prec@1 66.797 (67.288)
Epoch: [0][200/47380]	Time 1.705 (1.842)	Data 0.000 (0.007)	Loss 2.8215 (2.7392)	Prec@1 67.188 (67.135)
Epoch: [0][250/47380]	Time 1.702 (1.815)	Data 0.000 (0.006)	Loss 2.9494 (2.7502)	Prec@1 66.797 (67.122)
Epoch: [0][300/47380]	Time 1.705 (1.797)	Data 0.000 (0.005)	Loss 2.6762 (2.7560)	Prec@1 65.625 (67.017)
Epoch: [0][350/47380]	Time 1.699 (1.784)	Data 0.000 (0.004)	Loss 3.5162 (2.7578)	Prec@1 61.719 (67.027)
Epoch: [0][400/47380]	Time 1.709 (1.775)	Data 0.000 (0.004)	Loss 3.4846 (2.7631)	Prec@1 60.547 (66.979)
Epoch: [0][450/47380]	Time 1.712 (1.767)	Data 0.000 (0.004)	Loss 

Epoch: [0][3950/47380]	Time 1.708 (1.714)	Data 0.000 (0.001)	Loss 2.8374 (2.7611)	Prec@1 66.406 (66.886)
Epoch: [0][4000/47380]	Time 1.701 (1.714)	Data 0.000 (0.001)	Loss 3.2848 (2.7619)	Prec@1 59.375 (66.887)
Epoch: [0][4050/47380]	Time 1.704 (1.714)	Data 0.000 (0.001)	Loss 2.0281 (2.7622)	Prec@1 75.000 (66.880)
Epoch: [0][4100/47380]	Time 1.702 (1.714)	Data 0.000 (0.001)	Loss 2.7971 (2.7621)	Prec@1 70.703 (66.882)
Epoch: [0][4150/47380]	Time 1.703 (1.714)	Data 0.000 (0.001)	Loss 2.2905 (2.7618)	Prec@1 73.828 (66.886)
Epoch: [0][4200/47380]	Time 1.707 (1.714)	Data 0.000 (0.001)	Loss 2.5519 (2.7622)	Prec@1 68.359 (66.880)
Epoch: [0][4250/47380]	Time 1.707 (1.714)	Data 0.000 (0.001)	Loss 2.5170 (2.7615)	Prec@1 66.406 (66.883)
Epoch: [0][4300/47380]	Time 1.708 (1.714)	Data 0.000 (0.001)	Loss 2.7666 (2.7614)	Prec@1 63.672 (66.883)
Epoch: [0][4350/47380]	Time 1.701 (1.714)	Data 0.000 (0.001)	Loss 2.4322 (2.7608)	Prec@1 68.359 (66.886)
Epoch: [0][4400/47380]	Time 1.706 (1.714)	Data 0.000 (0

Epoch: [0][7900/47380]	Time 1.700 (1.712)	Data 0.000 (0.000)	Loss 2.9042 (2.7597)	Prec@1 68.359 (66.869)
Epoch: [0][7950/47380]	Time 1.709 (1.712)	Data 0.000 (0.000)	Loss 2.3226 (2.7595)	Prec@1 74.609 (66.871)
Epoch: [0][8000/47380]	Time 1.712 (1.712)	Data 0.000 (0.000)	Loss 3.6555 (2.7599)	Prec@1 60.938 (66.867)
Epoch: [0][8050/47380]	Time 1.708 (1.712)	Data 0.000 (0.000)	Loss 3.1459 (2.7601)	Prec@1 67.969 (66.862)
Epoch: [0][8100/47380]	Time 1.699 (1.712)	Data 0.000 (0.000)	Loss 2.5323 (2.7600)	Prec@1 68.750 (66.865)
Epoch: [0][8150/47380]	Time 1.711 (1.711)	Data 0.000 (0.000)	Loss 2.5488 (2.7602)	Prec@1 68.750 (66.863)
Epoch: [0][8200/47380]	Time 1.704 (1.711)	Data 0.000 (0.000)	Loss 2.8651 (2.7604)	Prec@1 66.797 (66.865)
Epoch: [0][8250/47380]	Time 1.703 (1.711)	Data 0.000 (0.000)	Loss 2.9671 (2.7606)	Prec@1 64.844 (66.864)
Epoch: [0][8300/47380]	Time 1.709 (1.711)	Data 0.000 (0.000)	Loss 3.1039 (2.7604)	Prec@1 63.672 (66.867)
Epoch: [0][8350/47380]	Time 1.703 (1.711)	Data 0.000 (0

Epoch: [0][11800/47380]	Time 1.702 (1.709)	Data 0.000 (0.000)	Loss 3.0203 (2.7586)	Prec@1 60.938 (66.829)
Epoch: [0][11850/47380]	Time 1.703 (1.709)	Data 0.000 (0.000)	Loss 2.9855 (2.7584)	Prec@1 64.062 (66.831)
Epoch: [0][11900/47380]	Time 1.705 (1.709)	Data 0.000 (0.000)	Loss 2.7393 (2.7583)	Prec@1 65.625 (66.829)
Epoch: [0][11950/47380]	Time 1.708 (1.709)	Data 0.000 (0.000)	Loss 2.8426 (2.7581)	Prec@1 67.969 (66.829)
Epoch: [0][12000/47380]	Time 1.705 (1.709)	Data 0.000 (0.000)	Loss 3.0594 (2.7583)	Prec@1 65.234 (66.827)
Epoch: [0][12050/47380]	Time 1.703 (1.709)	Data 0.000 (0.000)	Loss 2.8897 (2.7584)	Prec@1 63.672 (66.824)
Epoch: [0][12100/47380]	Time 1.703 (1.709)	Data 0.000 (0.000)	Loss 2.9991 (2.7584)	Prec@1 64.844 (66.825)
Epoch: [0][12150/47380]	Time 1.701 (1.709)	Data 0.000 (0.000)	Loss 2.5009 (2.7585)	Prec@1 67.969 (66.823)
Epoch: [0][12200/47380]	Time 1.698 (1.709)	Data 0.000 (0.000)	Loss 2.8382 (2.7583)	Prec@1 68.750 (66.825)
Epoch: [0][12250/47380]	Time 1.713 (1.709)	Dat

Epoch: [0][15700/47380]	Time 1.710 (1.708)	Data 0.000 (0.000)	Loss 3.1793 (2.7577)	Prec@1 61.328 (66.820)
Epoch: [0][15750/47380]	Time 1.701 (1.708)	Data 0.000 (0.000)	Loss 3.0843 (2.7576)	Prec@1 65.625 (66.821)
Epoch: [0][15800/47380]	Time 1.698 (1.708)	Data 0.000 (0.000)	Loss 2.4911 (2.7576)	Prec@1 74.609 (66.821)
Epoch: [0][15850/47380]	Time 1.709 (1.708)	Data 0.000 (0.000)	Loss 2.8284 (2.7574)	Prec@1 62.891 (66.823)
Epoch: [0][15900/47380]	Time 1.704 (1.708)	Data 0.000 (0.000)	Loss 2.2993 (2.7572)	Prec@1 69.531 (66.824)
Epoch: [0][15950/47380]	Time 1.698 (1.708)	Data 0.000 (0.000)	Loss 2.6625 (2.7572)	Prec@1 71.875 (66.825)
Epoch: [0][16000/47380]	Time 1.701 (1.708)	Data 0.000 (0.000)	Loss 2.8216 (2.7571)	Prec@1 63.281 (66.826)
Epoch: [0][16050/47380]	Time 1.705 (1.708)	Data 0.000 (0.000)	Loss 3.1270 (2.7570)	Prec@1 62.891 (66.826)
Epoch: [0][16100/47380]	Time 1.713 (1.708)	Data 0.000 (0.000)	Loss 2.6197 (2.7572)	Prec@1 67.188 (66.825)
Epoch: [0][16150/47380]	Time 1.709 (1.708)	Dat

...

Epoch: [2][31600/47380]	Time 1.705 (1.709)	Data 0.000 (0.000)	Loss 2.7308 (2.7562)	Prec@1 66.016 (66.823)
Epoch: [2][31650/47380]	Time 1.707 (1.709)	Data 0.000 (0.000)	Loss 2.5682 (2.7562)	Prec@1 64.453 (66.824)
Epoch: [2][31700/47380]	Time 1.704 (1.709)	Data 0.000 (0.000)	Loss 2.8479 (2.7563)	Prec@1 65.234 (66.822)
Epoch: [2][31750/47380]	Time 1.709 (1.709)	Data 0.000 (0.000)	Loss 2.4824 (2.7563)	Prec@1 70.703 (66.823)
Epoch: [2][31800/47380]	Time 1.709 (1.709)	Data 0.000 (0.000)	Loss 2.9340 (2.7562)	Prec@1 65.234 (66.824)
Epoch: [2][31850/47380]	Time 1.700 (1.709)	Data 0.000 (0.000)	Loss 2.9413 (2.7562)	Prec@1 64.453 (66.823)
Epoch: [2][31900/47380]	Time 1.705 (1.709)	Data 0.000 (0.000)	Loss 2.4765 (2.7563)	Prec@1 68.750 (66.823)
Epoch: [2][31950/47380]	Time 1.708 (1.709)	Data 0.000 (0.000)	Loss 3.2113 (2.7563)	Prec@1 66.016 (66.823)
Epoch: [2][32000/47380]	Time 1.705 (1.709)	Data 0.000 (0.000)	Loss 2.6718 (2.7563)	Prec@1 66.797 (66.823)
Epoch: [2][32050/47380]	Time 1.709 (1.709)	Dat

Epoch: [2][35500/47380]	Time 1.701 (1.709)	Data 0.000 (0.000)	Loss 2.8511 (2.7568)	Prec@1 67.969 (66.817)
Epoch: [2][35550/47380]	Time 1.702 (1.709)	Data 0.000 (0.000)	Loss 2.9575 (2.7569)	Prec@1 66.016 (66.818)
Epoch: [2][35600/47380]	Time 1.707 (1.709)	Data 0.000 (0.000)	Loss 2.8627 (2.7569)	Prec@1 65.625 (66.817)
Epoch: [2][35650/47380]	Time 1.725 (1.709)	Data 0.000 (0.000)	Loss 2.6380 (2.7569)	Prec@1 61.328 (66.818)
Epoch: [2][35700/47380]	Time 1.705 (1.709)	Data 0.000 (0.000)	Loss 2.7434 (2.7569)	Prec@1 69.922 (66.817)
Epoch: [2][35750/47380]	Time 1.706 (1.709)	Data 0.000 (0.000)	Loss 2.4835 (2.7570)	Prec@1 68.750 (66.817)
Epoch: [2][35800/47380]	Time 1.709 (1.709)	Data 0.000 (0.000)	Loss 2.3008 (2.7569)	Prec@1 69.922 (66.817)
Epoch: [2][35850/47380]	Time 1.699 (1.709)	Data 0.000 (0.000)	Loss 2.8643 (2.7571)	Prec@1 68.750 (66.817)
Epoch: [2][35900/47380]	Time 1.703 (1.709)	Data 0.000 (0.000)	Loss 2.3338 (2.7570)	Prec@1 70.703 (66.817)
Epoch: [2][35950/47380]	Time 1.712 (1.709)	Dat

Epoch: [2][39400/47380]	Time 1.709 (1.709)	Data 0.000 (0.000)	Loss 2.7331 (2.7571)	Prec@1 69.531 (66.816)
Epoch: [2][39450/47380]	Time 1.704 (1.709)	Data 0.000 (0.000)	Loss 2.9428 (2.7572)	Prec@1 66.406 (66.816)
Epoch: [2][39500/47380]	Time 1.741 (1.709)	Data 0.000 (0.000)	Loss 2.8919 (2.7572)	Prec@1 65.625 (66.816)
Epoch: [2][39550/47380]	Time 1.701 (1.709)	Data 0.000 (0.000)	Loss 2.5063 (2.7571)	Prec@1 71.094 (66.817)
Epoch: [2][39600/47380]	Time 1.711 (1.709)	Data 0.000 (0.000)	Loss 2.9782 (2.7571)	Prec@1 64.453 (66.818)
Epoch: [2][39650/47380]	Time 1.708 (1.709)	Data 0.000 (0.000)	Loss 2.3552 (2.7571)	Prec@1 69.141 (66.818)
Epoch: [2][39700/47380]	Time 1.706 (1.709)	Data 0.000 (0.000)	Loss 3.3750 (2.7572)	Prec@1 62.891 (66.818)
Epoch: [2][39750/47380]	Time 1.706 (1.709)	Data 0.000 (0.000)	Loss 2.7142 (2.7572)	Prec@1 67.969 (66.818)
Epoch: [2][39800/47380]	Time 1.709 (1.709)	Data 0.000 (0.000)	Loss 2.9098 (2.7572)	Prec@1 64.844 (66.818)
Epoch: [2][39850/47380]	Time 1.703 (1.709)	Dat

Epoch: [2][43300/47380]	Time 1.709 (1.709)	Data 0.000 (0.000)	Loss 2.7716 (2.7573)	Prec@1 69.141 (66.822)
Epoch: [2][43350/47380]	Time 1.703 (1.709)	Data 0.000 (0.000)	Loss 2.6903 (2.7573)	Prec@1 67.578 (66.822)
Epoch: [2][43400/47380]	Time 1.705 (1.709)	Data 0.000 (0.000)	Loss 2.5666 (2.7572)	Prec@1 69.922 (66.822)
Epoch: [2][43450/47380]	Time 1.704 (1.709)	Data 0.000 (0.000)	Loss 2.8271 (2.7572)	Prec@1 66.016 (66.822)
Epoch: [2][43500/47380]	Time 1.706 (1.709)	Data 0.000 (0.000)	Loss 3.3643 (2.7573)	Prec@1 60.938 (66.822)
Epoch: [2][43550/47380]	Time 1.702 (1.709)	Data 0.000 (0.000)	Loss 2.8684 (2.7573)	Prec@1 60.938 (66.821)
Epoch: [2][43600/47380]	Time 1.711 (1.709)	Data 0.000 (0.000)	Loss 2.8248 (2.7573)	Prec@1 66.406 (66.822)
Epoch: [2][43650/47380]	Time 1.711 (1.709)	Data 0.000 (0.000)	Loss 3.2912 (2.7573)	Prec@1 62.109 (66.822)
Epoch: [2][43700/47380]	Time 1.705 (1.709)	Data 0.000 (0.000)	Loss 2.6553 (2.7574)	Prec@1 67.578 (66.822)
Epoch: [2][43750/47380]	Time 1.712 (1.709)	Dat

Epoch: [2][47200/47380]	Time 1.711 (1.709)	Data 0.000 (0.000)	Loss 3.0232 (2.7568)	Prec@1 65.234 (66.827)
Epoch: [2][47250/47380]	Time 1.711 (1.709)	Data 0.000 (0.000)	Loss 3.0494 (2.7568)	Prec@1 65.234 (66.827)
Epoch: [2][47300/47380]	Time 1.711 (1.709)	Data 0.000 (0.000)	Loss 2.6612 (2.7567)	Prec@1 63.672 (66.827)
Epoch: [2][47350/47380]	Time 1.707 (1.709)	Data 0.000 (0.000)	Loss 2.1238 (2.7567)	Prec@1 75.781 (66.828)
Test: [0/946]	Time 3.336 (3.336)	Prec@1 80.859 (80.859)
Test: [20/946]	Time 1.347 (1.445)	Prec@1 83.594 (86.477)
Test: [40/946]	Time 1.350 (1.400)	Prec@1 34.375 (73.095)
Test: [60/946]	Time 1.346 (1.383)	Prec@1 95.312 (71.318)
Test: [80/946]	Time 1.347 (1.375)	Prec@1 80.859 (75.444)
Test: [100/946]	Time 1.460 (1.375)	Prec@1 80.469 (78.202)
Test: [120/946]	Time 1.347 (1.373)	Prec@1 88.281 (80.146)
Test: [140/946]	Time 1.353 (1.370)	Prec@1 88.281 (81.754)
Test: [160/946]	Time 1.348 (1.367)	Prec@1 92.188 (81.881)
Test: [180/946]	Time 1.345 (1.366)	Prec@1 3.125 (81.256)
Tes